In [3]:
import requests
from bs4 import BeautifulSoup

def get_tweet(url):
    headers = {
        'User-Agent': 'Twitterbot'
    }	
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        tweet_text = soup.find('meta', attrs={'property': 'og:description'}).get('content')
        if tweet_text:
            return tweet_text
        else:
            raise Exception('Error: Could not find tweet text')
    else:
        raise Exception('Error: Status code', response.status_code)

In [16]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

sarcasm_model = tf.keras.models.load_model('models/sarcasm/v1.h5')
positivity_model = tf.keras.models.load_model('models/positivity/v1.h5')

def predict_sarcasm(tweet):
    tweet = [tweet]
    sarcasm_tokenizer = Tokenizer(num_words=1000, oov_token='<OOV>')
    tweet = sarcasm_tokenizer.texts_to_sequences(tweet)
    print(tweet)
    tweet = pad_sequences(tweet, maxlen=16, padding='post', truncating='post')
    prediction = sarcasm_model.predict(tweet)
    return prediction[0][0]
    

In [17]:
TWEET_URL = 'https://twitter.com/lucas_montano/status/1775112771035701542'

tweet = get_tweet(TWEET_URL)

print(tweet)

sarcasm = predict_sarcasm(tweet)



agonia trabalhar com alguém de pé atrás
[[None, None, None, None, None, None, None]]


TypeError: int() argument must be a string, a bytes-like object or a real number, not 'NoneType'